Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [14]:
L2_weight = 0.5e-3
n_hidden_nodes = 1024
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  def linear(x, w, b):
    return tf.matmul(x, w) + b

  # Variables.
  with tf.name_scope('hidden'):
      weights_01 = tf.Variable(
        tf.truncated_normal([image_size * image_size, n_hidden_nodes]))
      biases_01 = tf.Variable(tf.zeros([n_hidden_nodes]))
      l2_reg_01 = tf.nn.l2_loss(weights_01)
    
  with tf.name_scope('output'):
    
      weights_12 = tf.Variable(
        tf.truncated_normal([n_hidden_nodes, num_labels]))
      biases_12 = tf.Variable(tf.zeros([num_labels]))
      l2_reg_12 = tf.nn.l2_loss(weights_12)
  
  def predict(input_x): 
    z_01 = linear(input_x, weights_01, biases_01)
    h1 = tf.nn.relu(z_01)
    logits = linear(h1, weights_12, biases_12)
    return logits
    
  def fit(data):
      train_logits = predict(data)
      loss = tf.reduce_mean(
          tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=train_logits))
      total_loss = loss + L2_weight * (l2_reg_01+l2_reg_12)
      return train_logits, total_loss

  train_logits, loss = fit(tf_train_dataset)
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
      


  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(train_logits)
  valid_prediction = tf.nn.softmax(predict(tf_valid_dataset))
  test_prediction = tf.nn.softmax(predict(tf_test_dataset))

In [17]:
num_steps = 3001
batch_size = 128


def train(train_size):
    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % train_size
        # Generate a minibatch.

        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
train(train_labels.shape[0] - batch_size)

Initialized
Minibatch loss at step 0: 429.018494
Minibatch accuracy: 14.1%
Validation accuracy: 25.4%
Minibatch loss at step 500: 150.146835
Minibatch accuracy: 73.4%
Validation accuracy: 78.7%
Minibatch loss at step 1000: 96.076500
Minibatch accuracy: 80.5%
Validation accuracy: 81.5%
Minibatch loss at step 1500: 75.196167
Minibatch accuracy: 82.0%
Validation accuracy: 81.7%
Minibatch loss at step 2000: 55.950993
Minibatch accuracy: 88.3%
Validation accuracy: 82.9%
Minibatch loss at step 2500: 43.528900
Minibatch accuracy: 90.6%
Validation accuracy: 84.0%
Minibatch loss at step 3000: 34.071129
Minibatch accuracy: 86.7%
Validation accuracy: 84.5%
Test accuracy: 91.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [19]:
train(batch_size*5)

Initialized
Minibatch loss at step 0: 613.153564
Minibatch accuracy: 7.0%
Validation accuracy: 27.3%
Minibatch loss at step 500: 122.623756
Minibatch accuracy: 100.0%
Validation accuracy: 75.8%
Minibatch loss at step 1000: 95.496506
Minibatch accuracy: 100.0%
Validation accuracy: 75.8%
Minibatch loss at step 1500: 74.370445
Minibatch accuracy: 100.0%
Validation accuracy: 75.7%
Minibatch loss at step 2000: 57.917953
Minibatch accuracy: 100.0%
Validation accuracy: 75.7%
Minibatch loss at step 2500: 45.105164
Minibatch accuracy: 100.0%
Validation accuracy: 75.7%
Minibatch loss at step 3000: 35.126888
Minibatch accuracy: 100.0%
Validation accuracy: 75.5%
Test accuracy: 83.4%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [37]:
L2_weight = 0.5e-3
n_hidden_nodes = 1024
batch_size = 128


graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  dropout_p = tf.placeholder(tf.float32)
  no_dropout = tf.constant(1.0, tf.float32)
  global_step = tf.Variable(0, trainable=False)  # count the number of steps taken.

  def linear(x, w, b):
    return tf.matmul(x, w) + b

  # Variables.
  with tf.name_scope('hidden'):
      weights_01 = tf.Variable(
        tf.truncated_normal([image_size * image_size, n_hidden_nodes]))
      biases_01 = tf.Variable(tf.zeros([n_hidden_nodes]))
      l2_reg_01 = tf.nn.l2_loss(weights_01)
    
  with tf.name_scope('output'):
    
      weights_12 = tf.Variable(
        tf.truncated_normal([n_hidden_nodes, num_labels]))
      biases_12 = tf.Variable(tf.zeros([num_labels]))
      l2_reg_12 = tf.nn.l2_loss(weights_12)
  
  def predict(input_x, dropout_p): 
    z_01 = linear(input_x, weights_01, biases_01)
    h1 = tf.nn.dropout(tf.nn.relu(z_01), dropout_p)
    logits = linear(h1, weights_12, biases_12)
    return logits
    
  def fit(data):
      train_logits = predict(data, dropout_p)
      loss = tf.reduce_mean(
          tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=train_logits))
      total_loss = loss + L2_weight * (l2_reg_01+l2_reg_12)
        
      
      
      
      return train_logits, total_loss

  train_logits, loss = fit(tf_train_dataset)

  learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.96)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)  


  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(train_logits)
  valid_prediction = tf.nn.softmax(predict(tf_valid_dataset, no_dropout))
  test_prediction = tf.nn.softmax(predict(tf_test_dataset, no_dropout))

In [26]:
num_steps = 3001
batch_size = 128


def train(train_size, checkpoint=0):
    with tf.Session(graph=graph) as session:

      tf.global_variables_initializer().run()

      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % train_size
        # Generate a minibatch.

        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, dropout_p:0.5}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
train(batch_size*5)

Initialized
Minibatch loss at step 0: 717.099915
Minibatch accuracy: 8.6%
Validation accuracy: 31.5%
Minibatch loss at step 500: 123.444893
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%
Minibatch loss at step 1000: 96.296509
Minibatch accuracy: 100.0%
Validation accuracy: 78.4%
Minibatch loss at step 1500: 75.134850
Minibatch accuracy: 100.0%
Validation accuracy: 77.6%
Minibatch loss at step 2000: 58.634392
Minibatch accuracy: 100.0%
Validation accuracy: 78.3%
Minibatch loss at step 2500: 45.782562
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Minibatch loss at step 3000: 35.757912
Minibatch accuracy: 100.0%
Validation accuracy: 78.4%
Test accuracy: 85.6%


In [39]:
num_steps = 10001
train(train_labels.shape[0] - batch_size)

Initialized
Minibatch loss at step 0: 717.110168
Minibatch accuracy: 9.4%
Validation accuracy: 25.4%
Minibatch loss at step 500: 170.114166
Minibatch accuracy: 73.4%
Validation accuracy: 80.5%
Minibatch loss at step 1000: 101.275696
Minibatch accuracy: 67.2%
Validation accuracy: 79.4%
Minibatch loss at step 1500: 82.181480
Minibatch accuracy: 77.3%
Validation accuracy: 80.2%
Minibatch loss at step 2000: 60.511639
Minibatch accuracy: 76.6%
Validation accuracy: 80.6%
Minibatch loss at step 2500: 47.042233
Minibatch accuracy: 80.5%
Validation accuracy: 81.9%
Minibatch loss at step 3000: 38.076504
Minibatch accuracy: 82.0%
Validation accuracy: 82.9%
Minibatch loss at step 3500: 29.736393
Minibatch accuracy: 85.9%
Validation accuracy: 83.6%
Minibatch loss at step 4000: 24.397171
Minibatch accuracy: 80.5%
Validation accuracy: 84.4%
Minibatch loss at step 4500: 19.572409
Minibatch accuracy: 81.2%
Validation accuracy: 85.1%
Minibatch loss at step 5000: 16.036076
Minibatch accuracy: 80.5%
Valid

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
